In [ ]:
from bs4 import BeautifulSoup
from contextlib import contextmanager
import csv
from datetime import datetime
import json
import numpy as np
import os
import pandas as pd
import pytz
import re
from urllib.request import urlopen
from urllib.request import urlretrieve
import threading
import timeit
import _thread

In [ ]:
print("Type car brand to scrap data from Allegro (select from: Audi/ BMW/ Skoda/ Honda/ Ford/ Mercedes/ Subaru/ Seat)")
car_model=input()
try_link="https://allegro.pl/kategoria/samochody-osobowe-4029?string="+car_model+"&bmatch=baseline-product-cl-eyesa2-engag-dict45-aut-1-4-0605"
try_link
type(try_link)
html = urlopen(try_link)
bs = BeautifulSoup(html, 'lxml')
for item in bs.findAll("a",href=True)[:3] :
    print(item, '\n')

In [ ]:
for item in bs.findAll("a", href=re.compile("allegro.pl/ogloszenie"))[:20]:
    print(item.attrs["href"])

In [ ]:
def get_page_offers(try_link):
    html = urlopen(try_link)
    bs = BeautifulSoup(html, 'lxml')
    offer_list = []
    for item in bs.findAll("a", href=re.compile("allegro.pl/ogloszenie")):
        offer_list.append(item.attrs["href"])
    offer_list = list(set(offer_list))
    return (offer_list)

In [ ]:
for item in bs.findAll("a", {"aria-label":re.compile("przejdź do strony")}):
    print(item.attrs["href"]) 

In [ ]:
for item in bs.findAll("a", {"aria-label":re.compile("następna")}):
    print(item.attrs["href"])

In [ ]:
def get_next_page(try_link):
    html = urlopen(try_link)
    bs = BeautifulSoup(html, 'lxml')
    next_page = bs.findAll("a", {"aria-label": re.compile("następna")})[0].attrs["href"]
    return (next_page)

In [ ]:
def get_n_following(try_link,n):
    page_list=[try_link]
    link_temp=try_link
    print("Page 0 is added to all pages list:\n{}\n".format(try_link))
    
    for i in range(0,n):
        try:
            next_page=get_next_page(link_temp)            
            print("Page {} added to total pages list:\n{}\n".format(i+1,next_page))            
            link_temp=next_page
            page_list.append(next_page)
        except:
            break
    return(page_list)

In [ ]:
page_list=get_n_following(try_link,3)
len(page_list)

In [ ]:
all_offers=[]
for i in range(0,len(page_list)-1):
    print("Extracting Allegro offers from selected link")
    print(page_list[i])
    all_offers=all_offers+get_page_offers(page_list[i])
    print("Total cumulative number of offers={}\n".format(len(all_offers)))

In [ ]:
offer=all_offers[6]
offer

In [ ]:
html = urlopen(offer)
bs = BeautifulSoup(html, 'lxml')
bs.title.string

In [ ]:
param=re.findall("\[([^]]+)\]",bs.script.string)[0]

In [ ]:
param

In [ ]:
paramDict=json.loads(params)

In [ ]:
Ticket=paramDict["idItem"]
Ticket

In [ ]:
Price=paramDict["price"]
Price

In [ ]:
OEMdetails=paramDict["headNavigation"]
OEMdetails

In [ ]:
OEMdetails=re.findall(("(?<=Samochody\|Osobowe)(?s)(.*$)"),OEMdetails)[0]
OEM, Model, Serie=(OEMdetails.split("|"))[1:]

In [ ]:
data={}

In [ ]:
data["OEM"],data["Model"],data["Serie"]=(OEMdetails.split("|"))[1:]

In [ ]:
data["Ticket"]=Ticket
data["Price"]=paramDict["price"]
data["Title"]=bs.title.string

In [ ]:
data

In [ ]:
dataDict=bs.findAll("script",{'data-serialize-box-id':'9xQB2iYQTKGcjrnbLd_GVA==kQ32FRhOQcqYmKhfpeU6lQ=='})[0].string
dataDict=json.loads(dataDict)
dataDict

In [ ]:
dataDict.keys()
dataDict["groups"][0].keys()

In [ ]:
dataDict["groups"][0]['singleValueParams']

In [ ]:
for i in range(0,len(dataDict["groups"][0]['singleValueParams'])):
    feature=dataDict["groups"][0]['singleValueParams'][i]["name"]
    value=dataDict["groups"][0]['singleValueParams'][i]["value"]["name"]
    data[feature]=value

In [ ]:
print(bs.title.string)
print("\n")
for i in range(0,len(dataDict["groups"][0]['singleValueParams'])):
    feature=dataDict["groups"][0]['singleValueParams'][i]["name"]
    value=dataDict["groups"][0]['singleValueParams'][i]["value"]["name"]
    print("{}={}".format(feature, value))

In [ ]:
def Extract_data(link):
    html = urlopen(link)
    bs = BeautifulSoup(html, 'lxml')
    
    # Extracting key offer parameters 
    
    key_data=re.findall("\[([^]]+)\]",bs.script.string)[0]
    key_data=json.loads(key_data)
    
    OEMdetails=key_data["headNavigation"]  
    OEMdetails=re.findall(("(?<=Samochody\|Osobowe)(?s)(.*$)"),OEMdetails)[0]
    
    dataDict={}
    
    if len((OEMdetails.split("|"))[1:])==3:
    
        dataDict["OEM"],dataDict["model"],dataDict["serie"]=(OEMdetails.split("|"))[1:]
    elif len((OEMdetails.split("|"))[1:])==2:
        dataDict["OEM"],dataDict["model"]=(OEMdetails.split("|"))[1:3]
    else:
        dataDict["OEM"]=(OEMdetails.split("|"))[1]
        
    dataDict["Ticket"]=key_data["idItem"]
    dataDict["Price"]=key_data["price"]    
    dataDict["Title"]=bs.title.string
    dataDict["Download_date"]=str(datetime.now())
    
    #Extracting offer details 
    
    details=bs.findAll("script",{'data-serialize-box-id':'9xQB2iYQTKGcjrnbLd_GVA==kQ32FRhOQcqYmKhfpeU6lQ=='})[0].string
    details=json.loads(details)
    offer_details=details["groups"][0]['singleValueParams']
    
    for i in range(0,len(offer_details)):
        feature=offer_details[i]["name"]
        value=offer_details[i]["value"]["name"]
        dataDict[feature]=value 
    return(dataDict)

In [ ]:
len(all_offers)

In [ ]:
offer=all_offers[0]
offer

In [ ]:
Extract_data(offer)

In [ ]:
class TimeoutException(Exception):
    def __init__(self, msg=''):
        self.msg = msg

@contextmanager
def time_limit(seconds, msg=''):
    timer = threading.Timer(seconds, lambda: _thread.interrupt_main())
    timer.start()
    try:
        yield
    except KeyboardInterrupt:
        pass
    finally:
        timer.cancel()

In [ ]:
len(all_offers)

In [ ]:
print(datetime.now(pytz.timezone('Europe/Amsterdam')))

In [ ]:
timestring=str(datetime.now(pytz.timezone('Europe/Amsterdam')))[11:19]
timestring

In [ ]:
timestring=timestring.replace(":","-")
timestring

In [ ]:
def Offer_database(offers_list,time,directory):  
    
    data=[]
    failed_extractions=0
    
    for i in range(0, len(all_offers)):
        
        #Each loops tries to extract data from one offer until "time" amount of seconds is exceeded. Exceeding time limit 
        #passes the loop to the next offer
        
        with time_limit(time, 'sleep'):
            try:
                offer=offers_list[i]
                if np.mod(i,100)==0:
                    print("Extraction: {}/{} at:{}".format(i+1,len(offers_list),str(datetime.now())[:-7] ))
                offer_data=Extract_data(offer)
                data.append(offer_data)
            except:
                print("Extraction: {}/{}  !!FAILED!!scrapping took too long".format(i+1,len(offers_list)))
                failed_extractions=failed_extractions+1
                pass
            
    # After scraping data the summary of results is printed.
    
    print("\n------------------------------------------\n")
    print("Sucessfully Extracted: {}/{} offers \nErrors:{}".format((i-failed_extractions+1),i+1,failed_extractions))
     
    # Data is converted from json to DataFrame and saved as a csv file in given directory
    
    data_json=json.dumps(data)
    df=pd.read_json(data_json, orient='records')
    timestring=str(datetime.now(pytz.timezone('Europe/Amsterdam')))[11:19]
    timestring=timestring.replace(":","-")
    timestring
    print("Saving {} offers at{}".format(i-failed_extractions+1,directory+"/"+timestring+".csv"))
   
    df.to_csv(directory+"/"+timestring+".csv")

In [ ]:
Offer_database(all_offers,5,'/home/monika/Pulpit/SCRAP/databaza_monika')

In [ ]:
def split_offers(offer_list, batch_size):
    offers_split=[offer_list[x:x+batch_size] for x in range(0, len(offer_list), batch_size)]
    return(offers_split)

In [ ]:
def Offer_database_batches(offers_list, time, batch_size, folder_name):
    import os
    todayDate = str(datetime.utcnow().date())
    directory = '/home/monika/Pulpit/SCRAP/database_allegro_' + folder_name + "_" + todayDate
    if not os.path.exists(directory):
        os.makedirs(directory)

    offers_split = split_offers(offers_list, batch_size)
    for i in range(0, len(offers_split)):
        print("Starting Batch no.{}/{}\n".format(i + 1, len(offers_split)))
        offers = offers_split[i]
        Offer_database(offers, time, directory)
        print("\n--------\nFinished Batch no.{}/{}\n\n".format(i + 1,len(offers_split)))

In [ ]:
car_model

In [ ]:
folder=str(car_model)+"_database"
folder

In [ ]:
Offer_database_batches(all_offers,20,100,folder)